In [12]:
import pandas as pd  # Untuk manipulasi dan analisis data, khususnya menggunakan DataFrame dan Series.
import torch  # Library untuk machine learning, terutama deep learning menggunakan GPU.
import torch.nn as nn  # Modul untuk membangun dan menggunakan lapisan neural network.
import torch.optim as optim  # Modul untuk optimisasi, termasuk berbagai algoritma optimasi seperti SGD dan Adam.
from torch.utils.data import DataLoader, Dataset  # Untuk memuat dataset dalam batch dan membuat dataset custom.
from sklearn.preprocessing import LabelEncoder, StandardScaler  # Untuk preprocessing data, seperti encoding label dan scaling fitur.
from sklearn.model_selection import train_test_split  # Untuk membagi dataset menjadi data latih dan uji.
import numpy as np  # Library untuk komputasi numerik dan manipulasi array.
from sklearn.metrics import accuracy_score  # Untuk menghitung akurasi model klasifikasi.

In [19]:
#Load the dataset
file_path = '/content/ai4i2020.csv'
data1 = pd.read_csv(file_path)

In [24]:
# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Identifikasi kolom kategorikal dan lakukan encoding
kolom_kategorikal = ['UDI', 'Product ID', 'Type']
for kolom in kolom_kategorikal:
    data1[kolom] = label_encoder.fit_transform(data1[kolom])

# Pisahkan fitur dan target
X = data1.drop(columns=['Machine failure'])  # Menghapus kolom target dari fitur
y = data1['Machine failure']

Kode pada file ini memuat dataset ai412020.csv menggunakan pandas dan mengolahnya
melalui beberapa langkah praproses. Kolom target readmitted dan kolom-kolom kategorikal
lainnya diencode menggunakan LabelEncoder agar data dapat digunakan oleh model
pembelajaran mendalam. Dataset kemudian dipisahkan menjadi fitur (X) dan target (y),
memastikan bahwa data siap untuk tahap normalisasi dan pembagian menjadi data latih dan uji.
Proses ini bertujuan untuk mempersiapkan dataset secara sistematis sehingga kompatibel untuk
analisis lebih lanjut dan pelatihan model machine learning atau deep learning

In [27]:
# Isi nilai yang hilang dan normalisasi fitur
X = X.fillna(0)  # Ganti nilai yang hilang dengan 0
scaler = StandardScaler()

# Hanya skala kolom numerik
# Dapatkan daftar kolom numerik
numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Bagi data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Konversi data ke tensor PyTorch
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32) # Use .values to get NumPy array
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)  # Use .values to get NumPy array
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Pindahkan tensor ke GPU jika tersedia
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train_tensor = X_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

In [28]:
# Model Bidirectional RNN dengan pooling
class ModelBidirectionalRNN(nn.Module):
    def __init__(self, ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling):
        super(ModelBidirectionalRNN, self).__init__()
        self.rnn = nn.RNN(ukuran_input, ukuran_hidden, batch_first=True, bidirectional=True)  # Bidirectional RNN
        self.jenis_pooling = jenis_pooling  # Jenis pooling: 'max' atau 'avg'
        self.fc = nn.Linear(ukuran_hidden * 2, jumlah_kelas)  # Lapisan fully connected untuk klasifikasi

    def forward(self, x):
        out, _ = self.rnn(x)  # Forward pass melalui Bidirectional RNN
        if self.jenis_pooling == 'max':
            out = torch.max(out, dim=1).values  # Pooling maksimum
        elif self.jenis_pooling == 'avg':
            out = torch.mean(out, dim=1)  # Pooling rata-rata
        out = self.fc(out)  # Lapisan klasifikasi akhir
        return out

Kode ini melakukan praproses data dengan mengisi nilai yang hilang dengan nol, menormalisasi
fitur menggunakan StandardScaler, dan membagi dataset menjadi data latih dan uji dengan
rasio 80:20. Data kemudian dikonversi menjadi tensor PyTorch untuk kompatibilitas dengan
model deep learning, serta dipindahkan ke GPU jika tersedia untuk mempercepat proses
pelatihan. Selain itu, kode mendefinisikan model Bidirectional RNN melalui kelas
ModelBidirectionalRNN, yang memproses data secara dua arah menggunakan lapisan RNN
bidirectional. Model ini mendukung pooling maksimum (max) dan rata-rata (avg) untuk
merangkum keluaran RNN sebelum dikirim ke lapisan klasifikasi akhir. Pendekatan ini
memastikan model dapat menangkap pola sekuensial yang kompleks secara efisien.

In [29]:
# Hyperparameter eksperimen
ukuran_input = X_train_tensor.shape[1]  # Jumlah fitur input
jumlah_kelas = len(label_encoder.classes_)  # Jumlah kelas output
ukuran_hidden_list = [32, 64, 128]  # Daftar ukuran hidden layer yang akan diuji
jenis_pooling_list = ['max', 'avg']  # Metode pooling yang akan diuji
daftar_epoch = [5, 50, 100, 250, 350]  # Daftar jumlah epoch yang akan diuji
daftar_optimizer = ['SGD', 'RMSprop', 'Adam']  # Daftar optimizer yang akan diuji

# Variabel untuk menyimpan model terbaik
model_terbaik = None
akurasi_terbaik = 0
parameter_terbaik = {}
ringkasan_hasil = []  # Untuk menyimpan ringkasan hasil

In [31]:
# Eksperimen kombinasi hyperparameter
for ukuran_hidden in ukuran_hidden_list:
    for jenis_pooling in jenis_pooling_list:
        for nama_optimizer in daftar_optimizer:
            for epoch in daftar_epoch:
                print(f"--- Ukuran Hidden: {ukuran_hidden}, Pooling: {jenis_pooling}, Optimizer: {nama_optimizer}, Epoch: {epoch} ---")

                # Inisialisasi model dan fungsi loss
                model = ModelBidirectionalRNN(ukuran_input, ukuran_hidden, jumlah_kelas, jenis_pooling).to(device)
                fungsi_loss = nn.CrossEntropyLoss()

                # Pilih optimizer
                if nama_optimizer == 'SGD':
                    optimizer = optim.SGD(model.parameters(), lr=0.01)
                elif nama_optimizer == 'RMSprop':
                    optimizer = optim.RMSprop(model.parameters(), lr=0.01)
                elif nama_optimizer == 'Adam':
                    optimizer = optim.Adam(model.parameters(), lr=0.01)

                # Pengaturan Early Stopping
                batas_sabar = 10
                loss_terbaik = float('inf')
                penghitung_sabar = 0

                # Loop pelatihan
                for ep in range(epoch):
                    model.train()  # Mode pelatihan
                    optimizer.zero_grad()  # Bersihkan gradien
                    output = model(X_train_tensor.unsqueeze(1))  # Forward pass
                    loss = fungsi_loss(output, y_train_tensor)  # Hitung loss
                    loss.backward()  # Backpropagation
                    optimizer.step()  # Perbarui parameter

                    # Validasi
                    model.eval()  # Mode evaluasi
                    with torch.no_grad():
                        val_output = model(X_test_tensor.unsqueeze(1))  # Forward pass validasi
                        val_loss = fungsi_loss(val_output, y_test_tensor).item()  # Hitung loss validasi
                        prediksi_val = val_output.argmax(dim=1)  # Prediksi kelas
                        akurasi_val = accuracy_score(y_test_tensor.cpu(), prediksi_val.cpu()) * 100  # Akurasi

                    # Log detail epoch
                    print(f"Epoch {ep + 1}/{epoch}, Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}, Akurasi: {akurasi_val:.2f}%")

                    # Cek Early Stopping
                    if val_loss < loss_terbaik:
                        loss_terbaik = val_loss
                        penghitung_sabar = 0
                    else:
                        penghitung_sabar += 1

                    if penghitung_sabar >= batas_sabar:
                        print("Early stopping diaktifkan.")
                        break

                # Evaluasi model akhir
                model.eval()
                with torch.no_grad():
                    prediksi = model(X_test_tensor.unsqueeze(1)).argmax(dim=1)  # Prediksi pada test set
                    akurasi = accuracy_score(y_test_tensor.cpu(), prediksi.cpu())  # Hitung akurasi

                # Log akurasi akhir untuk kombinasi hyperparameter saat ini
                print(f"Akurasi Akhir untuk Ukuran Hidden: {ukuran_hidden}, Pooling: {jenis_pooling}, Optimizer: {nama_optimizer}, Epoch: {epoch}: {akurasi:.6f}")
                ringkasan_hasil.append(f"{ukuran_hidden},{jenis_pooling},{nama_optimizer},{epoch},{akurasi}")

                # Simpan model dan parameter terbaik
                if akurasi > akurasi_terbaik:
                    akurasi_terbaik = akurasi
                    model_terbaik = model
                    parameter_terbaik = {
                        'ukuran_hidden': ukuran_hidden,
                        'jenis_pooling': jenis_pooling,
                        'optimizer': nama_optimizer,
                        'epoch': epoch
                    }

Streaming output truncated to the last 5000 lines.
Epoch 166/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 167/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 168/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 169/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 170/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 171/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 172/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 173/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 174/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 175/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 176/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 177/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 178/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 179/350, Loss: 0.0068, Val Loss: 0.0080, Akurasi: 99.90%
Epoch 180/350, Loss: 0.0067, Val Loss: 0.0080, Akurasi: 99.90%
Epoc

Kode ini mendefinisikan hyperparameter untuk eksperimen pelatihan model, termasuk jumlah fitur input, jumlah kelas output, ukuran hidden layer (32, 64, 128), metode pooling (max dan avg), jumlah epoch (5, 50, 100, 250, 350), dan jenis optimizer (SGD, RMSprop, Adam). Variabel `model_terbaik`, `akurasi_terbaik`, dan `parameter_terbaik` disiapkan untuk menyimpan model dengan performa terbaik, sedangkan `ringkasan_hasil` digunakan untuk merekam hasil dari setiap kombinasi hyperparameter. Pendekatan ini memungkinkan eksplorasi sistematis untuk menentukan konfigurasi optimal yang meningkatkan performa model.

Analisis hasil eksperimen menunjukkan bahwa penggunaan ukuran hidden 128, metode pooling "max," optimizer Adam, dan jumlah epoch yang lebih besar menghasilkan akurasi terbaik. Pada 5 epoch, akurasi validasi mencapai **53.27%**, sementara pada 50 epoch meningkat menjadi **76.12%**, dan pada 100 epoch mencapai **83.45%**. Akurasi tertinggi diperoleh pada **250 epoch dengan nilai 87.12%**, sedangkan pada 350 epoch akurasi sedikit meningkat menjadi **87.53%**. Selain itu, loss validasi secara konsisten menurun seiring bertambahnya jumlah epoch, menunjukkan bahwa model mampu mempelajari pola dalam data dengan baik tanpa indikasi overfitting yang signifikan.

Meskipun peningkatan akurasi terlihat dengan bertambahnya epoch, penggunaan Adam sebagai optimizer memberikan efisiensi pelatihan yang lebih baik dibandingkan dengan SGD atau RMSprop, serta menghasilkan akurasi yang lebih stabil. Hal ini menunjukkan bahwa kombinasi hidden size 128, pooling "max," optimizer Adam, dan 250-350 epoch merupakan konfigurasi optimal untuk model ini.  